# Winery classification with the multivariate Gaussian

In this notebook, we return to winery classification, using the full set of 13 features.

## 1. Load in the data 

As usual, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [1]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 

In [2]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

In [3]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [80]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)
x = np.ones(13)
label = 1
features = [1, 4, 3]
#sigma_simple = [[1,0.2], [0.5,1]]
s = np.zeros([13,13])
dimension = 13
dim = np.arange(0,dimension)
for i in dim :
    for j in dim:
        if i in features and j in features:
            s[i,j] = 1
s = s.astype(bool)
#print(mu.shape)
sigma_simple = sigma[label, s]
sigma_simple = np.reshape(sigma_simple, [len(features), len(features)])
sign, det = np.linalg.slogdet(sigma_simple)
score = (0.5*(x[features]-mu[label, features])*sigma_simple*(x[features]-mu[label, features]).transpose()) + 0.5*len(features)*np.log(6.28) + sign*det
print(score)

[[ 8.64287842e+00  1.31720426e+03 -2.23582542e+01]
 [ 8.54193268e+00  3.32014095e+04  7.09848781e+02]
 [ 8.28994734e+00  3.05744263e+04  1.49475681e+04]]


In [81]:
dim = np.arange(0,12)
d = dim+ dim.transpose()
print(dim.shape)

(12,)


## 3. Use the model to make predictions on the test set

<font color="magenta">**For you to do**</font>: Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It should return the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so 

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)` 

should print 7/48.

**Hint:** The way you restrict attention to a subset of features is by choosing the corresponding coordinates of the full 13-dimensional mean and the appropriate submatrix of the full 13x13 covariance matrix.

In [235]:
# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    ###
    ### Your code goes here   
    #score = np.zeros(len(tx))
    #mu = mu.transpose()
    mean = mu[:, features]
    s = np.zeros([4,len(featurenames), len(featurenames)])
    #print(s.shape)
    for i in np.arange(1, len(pi)):
        for j in np.arange(0, len(featurenames)):
            for k in np.arange(0, len(featurenames)):
                if j in features and k in features:
                    s[i, j, k] = 1
    s = s.astype(bool)
    sig = np.zeros([4,len(features), len(features)])
    #print(sigma[1,s[1]].shape)
    n = len(features)
    sig[1] = sigma[1,s[1]].reshape([n,n])
    sig[2] = sigma[2,s[2]].reshape([n,n])
    sig[3] = sigma[3,s[3]].reshape([n,n])
    score = np.zeros([4,len(tx)])
    #print(sig[1])
    for label in np.arange(1, len(pi)):
        sig_in = np.linalg.inv(sig[label])
        exp_vecset = tx[:,features] - mean[label]
        #print(exp_vecset)
        for i in np.arange(0, len(tx)):
            ex = exp_vecset[i].reshape([n,1])
            #print(sig_in*ex)
            #print(0.5*np.log(pi[label])*exp_vecset[i]*sig_in*ex)
            score[label, i] = 0.5*np.log(pi[label])*(np.dot(exp_vecset[i],sig_in.dot(exp_vecset[i])))
            
    predictions = np.zeros(len(tx))
    scor = score[1:]
    for i in np.arange(0, len(tx)):
        comp = scor[:, i]
        predictions[i] = np.argmax(comp) + 1
    return len(ty[ty!=predictions])
    ###

### <font color="magenta">Fast exercises</font>

*Note down the answers to these questions. You will need to enter them as part of this week's assignment.*

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [236]:
test_model(mu, sigma, pi, [2], testx, testy)

28

Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [237]:
test_model(mu, sigma, pi, [0,2], testx, testy)

14

Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [241]:
test_model(mu, sigma, pi, [1,3,5], testx, testy)

16

Exercise 4. How many errors when using all 13 features?

In [239]:
test_model(mu, sigma, pi, range(0,13), testx, testy)

2

Exercise 5. In lecture, we got somewhat different answers to these questions. Why do you think that might be?